### Задание

Создайте систему компьютерного зрения, которая будет определять тип геометрической фигуры. Используя подготовленную базу и шаблон ноутбука проведите серию экспериментов по перебору гиперпараметров нейронной сети, распознающей три категории изображений (треугольник, круг, квадрат).

1. Поменяйте количество нейронов в сети, используя следующие значения:

- один слой 10 нейронов
- один слой 100 нейронов
- один слой 5000 нейронов.

2. Поменяйте активационную функцию в скрытых слоях с `relu` на `linear`.
3. Поменяйте размеры batch_size:
- 10
- 100
- 1000

4. Выведите на экран получившиеся точности.

Всего должно получиться 18 комбинаций указанных параметров.

Создайте сравнительную таблицу по результатам проведенных тестов.

In [ ]:
# Подключение класса для создания нейронной сети прямого распространения
from tensorflow.keras.models import Sequential
# Подключение класса для создания полносвязного слоя
from tensorflow.keras.layers import Dense, Flatten
# Подключение оптимизатора
from tensorflow.keras.optimizers import Adam
# Подключение утилит для to_categorical
from tensorflow.keras import utils
# Подключение библиотеки для загрузки изображений
from tensorflow.keras.preprocessing import image
# Подключение библиотеки для работы с массивами
import numpy as np
# Подключение библиотек для отрисовки изображений
import matplotlib.pyplot as plt
# Подключение модуля для работы с файлами
import os
# Вывод изображения в ноутбуке, а не в консоли или файле
%matplotlib inline

In [ ]:
# Загрузка датасета из облака
import gdown
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l3/hw_light.zip', None, quiet=True)

'hw_light.zip'

In [ ]:
# Распаковываем архив hw_light.zip в папку hw_light
!unzip -q hw_light.zip

In [ ]:
# Путь к директории с базой
base_dir = '/content/hw_light'
# Создание пустого списка для загрузки изображений обучающей выборки
x_train = []
# Создание списка для меток классов
y_train = []
# Задание высоты и ширины загружаемых изображений
img_height = 20
img_width = 20
# Перебор папок в директории базы
for patch in os.listdir(base_dir):
    # Перебор файлов в папках
    for img in os.listdir(base_dir + '/' + patch):
        # Добавление в список изображений текущей картинки
        x_train.append(image.img_to_array(image.load_img(base_dir + '/' + patch + '/' + img,
                                                    target_size=(img_height, img_width),
                                                    color_mode='grayscale')))
        # Добавление в массив меток, соответствующих классам
        if patch == '0':
            y_train.append(0)
        elif patch == '3':
            y_train.append(1)
        else:
            y_train.append(2)

# Преобразование в numpy-массив загруженных изображений и меток классов
x_train = np.array(x_train)
y_train = np.array(y_train)
# Вывод размерностей
print('Размер массива x_train', x_train.shape)
print('Размер массива y_train', y_train.shape)

Размер массива x_train (302, 20, 20, 1)
Размер массива y_train (302,)


In [ ]:
# Преобразуем тип данных и нормируем:
x_train = x_train.astype('float32') / 255.0
if len(x_train.shape) == 3:
    x_train = np.expand_dims(x_train, axis=-1)

# Преобразуем y_train в one-hot вектор для 3 классов
y_train = utils.to_categorical(y_train, 3)

In [ ]:
def net_model(neurons, activation_f, batch_s):
  """
  Создание сети по параметрам.
  """
  # Теперь создадим нейронную сеть
  model = Sequential()
  model.add(Flatten(input_shape=(20, 20, 1)))
  model.add(Dense(neurons, activation=activation_f))

  # Выходной слой на 3 нейрона (по числу классов) с softmax-активацией
  model.add(Dense(3, activation='softmax'))

  # Компиляция модели
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  # Обучение
  history = model.fit(x_train, y_train, epochs=10, batch_size=batch_s, verbose=1)
  # Смотрим итоговую точность
  train_acc = history.history['accuracy'][-1]
  print('Для модели с: нейронов =', neurons, 'активацией:', activation_f, 'батчами =', batch_s)
  print(f"Точность: {train_acc:.3f}")

net_model(10, 'relu', 10)
net_model(10, 'relu', 100)
net_model(10, 'relu', 1000)
net_model(10, 'linear', 10)
net_model(10, 'linear', 100)
net_model(10, 'linear', 1000)
net_model(100, 'relu', 10)
net_model(100, 'relu', 100)
net_model(100, 'relu', 1000)
net_model(100, 'linear', 10)
net_model(100, 'linear', 100)
net_model(100, 'linear', 1000)
net_model(5000, 'relu', 10)
net_model(5000, 'relu', 100)
net_model(5000, 'relu', 1000)
net_model(5000, 'linear', 10)
net_model(5000, 'linear', 100)
net_model(5000, 'linear', 1000)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.2581 - loss: 1.7657
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4213 - loss: 1.1030
Epoch 3/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4618 - loss: 1.0243
Epoch 4/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6330 - loss: 0.8698
Epoch 5/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6905 - loss: 0.7822
Epoch 6/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7470 - loss: 0.7339
Epoch 7/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7741 - loss: 0.6623
Epoch 8/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7522 - loss: 0.6622
Epoch 9/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7969 - loss: 0.6323
Epoch 10/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7673 - loss: 0.6232
Для модели с: нейронов =  10 активацией:  relu батчами =  10
Точность: 0.811
Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step - accuracy: 0.3430 - loss: 1.1083
Epoch 2/